In [1]:
import pandas as pd
import numpy as np

from fit_adversarial import fit_adversarial_regressor, test_model
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("all_vals_race1_interestrate1.csv")
from sklearn.model_selection import train_test_split

df.drop(
    columns=[
        "IsTestData",
        "IsCalibrateData",
        "v1",
        "istestdata2",
        "iscalibratedata2",
    ],
    inplace=True,
)


df["minority"] = (
    (df["race_dum_Black"] == 1)
    | (df["race_dum_Native Am, Alaska, Hawaii"] == 1)
    | (df["race_dum_White hisp"] == 1)
)

C:\Users\bencu\AppData\Local\Temp\ipykernel_22148\3910315459.py:7: DtypeWarning: Columns (101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_vals_race1_interestrate1.csv")


In [2]:
import tensorflow as tf

print(tf.sysconfig.get_build_info())
print("GPUs Available:", tf.config.list_physical_devices("GPU"))

OrderedDict([('cpu_compiler', 'C:/Program Files (x86)/Microsoft Visual Studio/2019/Community/VC/Tools/MSVC/14.29.30133/bin/HostX64/x64/cl.exe'), ('cuda_compute_capabilities', ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']), ('cuda_version', '64_112'), ('cudart_dll_name', 'cudart64_112.dll'), ('cudnn_dll_name', 'cudnn64_8.dll'), ('cudnn_version', '64_8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', False), ('msvcp_dll_names', 'msvcp140.dll,msvcp140_1.dll'), ('nvcuda_dll_name', 'nvcuda.dll')])
GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
race_columns = [
    "race_dum_Unknown",
    "race_dum_Asian",
    "race_dum_Black",
    "race_dum_Native Am, Alaska, Hawaii",
    "race_dum_White hisp",
    "race_dum_White_non_hisp",
]

ethnicity_column = "ethnicity_dum_hisp"

features = df.drop(
    columns=["default", "cur_int_rate", "sato", "default_age"] + race_columns + [ethnicity_column],
    axis=1,
)

target = df["cur_int_rate"]


sensitive_features = df[["minority"]].values

import math

features = features.applymap(
    lambda x: x if isinstance(x, (int, float)) and not math.isnan(x) else 0
)

C:\Users\bencu\AppData\Local\Temp\ipykernel_22148\1955962376.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features = features.applymap(


In [4]:
X_train, X_val, y_train, y_val, sensitive_train, sensitive_val = train_test_split(
    features, target, sensitive_features, test_size=0.3, random_state=42
)

In [5]:
mitigator0, scaler0 = fit_adversarial_regressor(
    alpha=0,
    X_train=X_train,
    X_val=X_val,
    y_train=y_train,
    y_val=y_val,
    sensitive_train=sensitive_train,
    sensitive_val=sensitive_val,
    filename="models/alpha0.pkl",
    epochs=20,
    use_skip_connections=False,
    update_alpha=None,
    update_epoch=11,
    learning_rate_decay=.95
)

Epoch 1/20, Average adversary loss: 0.503230. Average predictor loss: 0.002213
Current alpha 0, learning_rate 0.00095
Validation - Epoch 1/20, Adversary Loss: 5.111218, Predictor Loss: 0.000065
Epoch 2/20, Average adversary loss: 0.501435. Average predictor loss: 0.000103
Current alpha 0, learning_rate 0.0009025
Validation - Epoch 2/20, Adversary Loss: 5.267409, Predictor Loss: 0.000070
Epoch 3/20, Average adversary loss: 0.501669. Average predictor loss: 0.000093
Current alpha 0, learning_rate 0.000857375
Validation - Epoch 3/20, Adversary Loss: 5.058170, Predictor Loss: 0.000058
Epoch 4/20, Average adversary loss: 0.501465. Average predictor loss: 0.000076
Current alpha 0, learning_rate 0.0008145062499999999
Validation - Epoch 4/20, Adversary Loss: 5.082978, Predictor Loss: 0.000065
Epoch 5/20, Average adversary loss: 0.502509. Average predictor loss: 0.000709
Current alpha 0, learning_rate 0.0007737809374999998
Validation - Epoch 5/20, Adversary Loss: 5.065942, Predictor Loss: 0.000

c:\Projects\fairlearn-adversarial\fairlearn\adversarial\_adversarial_mitigation.py:603: UserWarning: update_epoch was provided, but update_alpha was not. Alpha not updated.
  warnings.warn(


Validation - Epoch 11/20, Adversary Loss: 5.019497, Predictor Loss: 0.000051
Epoch 12/20, Average adversary loss: 0.499258. Average predictor loss: 0.000052
Current alpha 0, learning_rate 0.0005403600876626365
Validation - Epoch 12/20, Adversary Loss: 5.034309, Predictor Loss: 0.000051
Epoch 13/20, Average adversary loss: 0.498990. Average predictor loss: 0.000052
Current alpha 0, learning_rate 0.0005133420832795047
Validation - Epoch 13/20, Adversary Loss: 4.984026, Predictor Loss: 0.000050
Epoch 14/20, Average adversary loss: 0.498659. Average predictor loss: 0.000051
Current alpha 0, learning_rate 0.00048767497911552944
Validation - Epoch 14/20, Adversary Loss: 4.998568, Predictor Loss: 0.000050
Epoch 15/20, Average adversary loss: 0.498311. Average predictor loss: 0.000051
Current alpha 0, learning_rate 0.00046329123015975297
Validation - Epoch 15/20, Adversary Loss: 4.973956, Predictor Loss: 0.000050
Epoch 16/20, Average adversary loss: 0.498511. Average predictor loss: 0.000051
C

In [6]:
test_model(mitigator0, scaler0, features, target, sensitive_features)

{'mse': 4.896449547049055e-05,
 'rmse': 0.006997463502619399,
 'mae': 0.005147448430099457,
 'r2': 0.7663693251459245,
 'fairness_metrics': {'mean_metrics': {'independence': 0.0055226830958322575,
   'separation': 0.0010573537764778757,
   'sufficiency': 1.4241120988747514e-05},
  'std_metrics': {'independence': 5.0826480655298736e-05,
   'separation': 2.5308513221918054e-05,
   'sufficiency': 2.5335543615136633e-06}}}

In [7]:
mitigator1, scaler1 = fit_adversarial_regressor(
    alpha=0,
    X_train=X_train,
    X_val=X_val,
    y_train=y_train,
    y_val=y_val,
    sensitive_train=sensitive_train,
    sensitive_val=sensitive_val,
    filename="models/alpha0.pkl",
    epochs=20,
    use_skip_connections=False,
    update_alpha=1,
    update_epoch=11,
    learning_rate_decay=.95
)

Epoch 1/20, Average adversary loss: 0.502964. Average predictor loss: 0.001079
Current alpha 0, learning_rate 0.00095
Validation - Epoch 1/20, Adversary Loss: 5.117066, Predictor Loss: 0.000073
Epoch 2/20, Average adversary loss: 0.501540. Average predictor loss: 0.000102
Current alpha 0, learning_rate 0.0009025
Validation - Epoch 2/20, Adversary Loss: 5.153566, Predictor Loss: 0.000063
Epoch 3/20, Average adversary loss: 0.501482. Average predictor loss: 0.000085
Current alpha 0, learning_rate 0.000857375
Validation - Epoch 3/20, Adversary Loss: 5.188975, Predictor Loss: 0.000059
Epoch 4/20, Average adversary loss: 0.501343. Average predictor loss: 0.000067
Current alpha 0, learning_rate 0.0008145062499999999
Validation - Epoch 4/20, Adversary Loss: 5.050590, Predictor Loss: 0.000057
Epoch 5/20, Average adversary loss: 0.501345. Average predictor loss: 0.000062
Current alpha 0, learning_rate 0.0007737809374999998
Validation - Epoch 5/20, Adversary Loss: 5.119356, Predictor Loss: 0.000

In [8]:
test_model(mitigator1, scaler1, features, target, sensitive_features)

{'mse': 0.00019461694665487463,
 'rmse': 0.013950517791640374,
 'mae': 0.00988438743098514,
 'r2': 0.07139881360729028,
 'fairness_metrics': {'mean_metrics': {'independence': 0.0024316556635342055,
   'separation': 0.014351048309963334,
   'sufficiency': 0.016379100610190194},
  'std_metrics': {'independence': 2.2176730357656856e-05,
   'separation': 7.050345503778103e-05,
   'sufficiency': 8.15373987162778e-05}}}

In [9]:
mitigator2, scaler2 = fit_adversarial_regressor(
    alpha=0,
    X_train=X_train,
    X_val=X_val,
    y_train=y_train,
    y_val=y_val,
    sensitive_train=sensitive_train,
    sensitive_val=sensitive_val,
    filename="models/alpha0.pkl",
    epochs=20,
    use_skip_connections=False,
    update_alpha=2,
    update_epoch=11,
    learning_rate_decay=.95
)

Epoch 1/20, Average adversary loss: 0.503605. Average predictor loss: 0.001744
Current alpha 0, learning_rate 0.00095
Validation - Epoch 1/20, Adversary Loss: 5.105571, Predictor Loss: 0.000068
Epoch 2/20, Average adversary loss: 0.501573. Average predictor loss: 0.000123
Current alpha 0, learning_rate 0.0009025
Validation - Epoch 2/20, Adversary Loss: 5.097412, Predictor Loss: 0.000063
Epoch 3/20, Average adversary loss: 0.501520. Average predictor loss: 0.000085
Current alpha 0, learning_rate 0.000857375
Validation - Epoch 3/20, Adversary Loss: 5.069476, Predictor Loss: 0.000058
Epoch 4/20, Average adversary loss: 0.501546. Average predictor loss: 0.000072
Current alpha 0, learning_rate 0.0008145062499999999
Validation - Epoch 4/20, Adversary Loss: 5.131229, Predictor Loss: 0.000066
Epoch 5/20, Average adversary loss: 0.500972. Average predictor loss: 0.000065
Current alpha 0, learning_rate 0.0007737809374999998
Validation - Epoch 5/20, Adversary Loss: 5.067462, Predictor Loss: 0.000

In [10]:
test_model(mitigator2, scaler2, features, target, sensitive_features)

{'mse': 0.018738638139764403,
 'rmse': 0.13688914544172012,
 'mae': 0.12248629523669294,
 'r2': -88.41010486012094,
 'fairness_metrics': {'mean_metrics': {'independence': 0.5380937584828158,
   'separation': 0.5413298051503974,
   'sufficiency': 0.011460481885202833},
  'std_metrics': {'independence': 0.0003482008244785015,
   'separation': 0.0003571629449614351,
   'sufficiency': 9.501803155046984e-05}}}

In [11]:
mitigator5, scaler5 = fit_adversarial_regressor(
    alpha=0,
    X_train=X_train,
    X_val=X_val,
    y_train=y_train,
    y_val=y_val,
    sensitive_train=sensitive_train,
    sensitive_val=sensitive_val,
    filename="models/alpha0.pkl",
    epochs=20,
    use_skip_connections=False,
    update_alpha=5,
    update_epoch=11,
    learning_rate_decay=0.95,
)

Epoch 1/20, Average adversary loss: 0.503574. Average predictor loss: 0.002784
Current alpha 0, learning_rate 0.00095
Validation - Epoch 1/20, Adversary Loss: 5.128636, Predictor Loss: 0.000092
Epoch 2/20, Average adversary loss: 0.501567. Average predictor loss: 0.000119
Current alpha 0, learning_rate 0.0009025
Validation - Epoch 2/20, Adversary Loss: 5.085992, Predictor Loss: 0.000066
Epoch 3/20, Average adversary loss: 0.500646. Average predictor loss: 0.000148
Current alpha 0, learning_rate 0.000857375
Validation - Epoch 3/20, Adversary Loss: 5.123931, Predictor Loss: 0.000063
Epoch 4/20, Average adversary loss: 0.501158. Average predictor loss: 0.000076
Current alpha 0, learning_rate 0.0008145062499999999
Validation - Epoch 4/20, Adversary Loss: 5.093513, Predictor Loss: 0.000060
Epoch 5/20, Average adversary loss: 0.501409. Average predictor loss: 0.000098
Current alpha 0, learning_rate 0.0007737809374999998
Validation - Epoch 5/20, Adversary Loss: 5.064493, Predictor Loss: 0.000

In [12]:
test_model(mitigator5, scaler5, features, target, sensitive_features)

{'mse': 0.0001551539468915278,
 'rmse': 0.012456080719533243,
 'mae': 0.010012869596213319,
 'r2': 0.25969376442595904,
 'fairness_metrics': {'mean_metrics': {'independence': 0.001492198280831512,
   'separation': 0.01714905616181181,
   'sufficiency': 0.020094176637654806},
  'std_metrics': {'independence': 2.7096633017098384e-05,
   'separation': 0.00011267676114632943,
   'sufficiency': 0.00010503524861942474}}}

In [13]:
mitigator10, scaler10 = fit_adversarial_regressor(
    alpha=0,
    X_train=X_train,
    X_val=X_val,
    y_train=y_train,
    y_val=y_val,
    sensitive_train=sensitive_train,
    sensitive_val=sensitive_val,
    filename="models/alpha0.pkl",
    epochs=20,
    use_skip_connections=False,
    update_alpha=10,
    update_epoch=11,
    learning_rate_decay=0.95,
)

Epoch 1/20, Average adversary loss: 0.501654. Average predictor loss: 0.002221
Current alpha 0, learning_rate 0.00095
Validation - Epoch 1/20, Adversary Loss: 5.093400, Predictor Loss: 0.000066
Epoch 2/20, Average adversary loss: 0.501078. Average predictor loss: 0.000108
Current alpha 0, learning_rate 0.0009025
Validation - Epoch 2/20, Adversary Loss: 5.989422, Predictor Loss: 0.000203
Epoch 3/20, Average adversary loss: 0.501167. Average predictor loss: 0.000095
Current alpha 0, learning_rate 0.000857375
Validation - Epoch 3/20, Adversary Loss: 5.124606, Predictor Loss: 0.000068
Epoch 4/20, Average adversary loss: 0.501082. Average predictor loss: 0.000068
Current alpha 0, learning_rate 0.0008145062499999999
Validation - Epoch 4/20, Adversary Loss: 5.104486, Predictor Loss: 0.000061
Epoch 5/20, Average adversary loss: 0.500507. Average predictor loss: 0.000062
Current alpha 0, learning_rate 0.0007737809374999998
Validation - Epoch 5/20, Adversary Loss: 5.151113, Predictor Loss: 0.000

In [14]:
test_model(mitigator10, scaler10, features, target, sensitive_features)

{'mse': 0.007186830237677625,
 'rmse': 0.08477517465436225,
 'mae': 0.06850508259693247,
 'r2': -33.29145919622968,
 'fairness_metrics': {'mean_metrics': {'independence': 0.9992069402399663,
   'separation': 0.9991124421236254,
   'sufficiency': -0.114256338717633},
  'std_metrics': {'independence': 1.52716483094815e-05,
   'separation': 1.396917714494043e-05,
   'sufficiency': 0.010376130328498889}}}